In [48]:
indir = "/data1/peterr/bilingual_data_for_ABClf_output"
outdir = "/data1/peterr/bilingual_data_for_ABClf_output/with_domain_level"
import os
import pandas as pd
from dask import dataframe

all_files = os.listdir(indir)
f = all_files[0]
for f in all_files:
    if os.path.isdir(os.path.join(indir,f)):
        continue
    endpath = os.path.join(
            outdir,
            f
        )
    if os.path.exists(endpath):
        continue
    df = pd.read_csv(
        os.path.join(
            indir,
            f
        )
    )

    counts = df.domain.value_counts()

    # Use only domains where we have >= 3 docs
    suitable_domains = counts[counts >= 3].index.values

    # Default domain level is UNK unless we find otherwise
    df["domain_level_variant"] = "UNK"
    for domain in suitable_domains:
        # Calculate a subset of all docs that fit in this domain:
        subset = df.loc[df.domain == domain, :]

        # Disputed stuff:
        # Take the most frequent of either A, B, or MIX

        
        most_frequent = subset.variant_unbalanced.value_counts().index[0]

        df.loc[df.domain == domain, "domain_level_variant"] = most_frequent

    df.to_csv(
        os.path.join(
            outdir,
            f
        ),
        index=False
    )

# Logic restated

If a domain has at least 3 instances present, we can assign the majority variant to all of the instances of that domain. If we have even number of instances with equal A-B counts, we decide randomly. At this point we do not care about what to do with MIX.

Let's find suitable domains:

In [44]:
counts = df.domain.value_counts()
suitable_domains = counts[counts >= 3].index.values

In [45]:
df["domain_level_variant"] = "UNK"
for domain in suitable_domains:
    subset = df.loc[df.domain == domain, :]
    most_frequent = subset.variant_unbalanced.value_counts().index[0]
    df.loc[df.domain == domain, "domain_level_variant"] = most_frequent

df.to_csv(
    os.path.join(
        outdir,
        f
    ),
    index=False
)

In [49]:
df

,url,text,variant_unbalanced,per_token_unbalanced,variant_balanced,per_token_balanced,domain,domain_level_variant
0,http://1-----a-oabya.hotels-tokyo-jp.com/en/,"All accommodation units feature WiFi, a multi-...",UNK,{},UNK,{},1-----a-oabya.hotels-tokyo-jp.com,UNK
1,http://1-bedroom-apartment-in-taksim.hotel-ist...,"Located next to Demiroren Shopping Mall, the p...",B,"{'travellers': {'variant': 'B', 'count': 2}, '...",B,"{'travellers': {'variant': 'B', 'count': 2}, '...",1-bedroom-apartment-in-taksim.hotel-istanbul.net,UNK
2,http://1-bus-stop-to-nyc-ewr-airport.hotelinew...,"For your comfort, you will find a hairdryer, b...",A,"{'center': {'variant': 'A', 'count': 2}}",A,"{'center': {'variant': 'A', 'count': 2}}",1-bus-stop-to-nyc-ewr-airport.hotelinewyork.com,UNK
3,http://1-hotel-central-park.hotelinewyork.com/,1 Hotel Central Park New York is situated in v...,UNK,{},UNK,{},1-hotel-central-park.hotelinewyork.com,UNK
4,http://1.test.info.tr/kategori2_30_1_avukatlar,Sed non mauris vitae erat consequat auctor eu ...,UNK,{},UNK,{},1.test.info.tr,UNK
...,...,...,...,...,...,...,...,...
1517090,https://zzhu17.medium.com/followers?source=abo...,Business Leader who excels at the nexus of Dat...,UNK,{},UNK,{},zzhu17.medium.com,UNK
1517091,https://zzhu17.medium.com/followers?source=pos...,Ph.D. in Economics | Certified in Data Science...,UNK,{},UNK,{},zzhu17.medium.com,UNK
1517092,https://zzhu17.medium.com/followers?source=use...,Business Leader who excels at the nexus of Dat...,UNK,{},UNK,{},zzhu17.medium.com,UNK
1517093,https://zzhu17.medium.com/followers?source=use...,Ph.D. in Economics | Certified in Data Science...,UNK,{},UNK,{},zzhu17.medium.com,UNK


In [1]:
for i in range(10):
    print(i, end="\r")